Dane pobrane ze strony https://www.kaggle.com/c/tabular-playground-series-mar-2021/data
Ponieważ jak się okazuje algorytm lgbm dość dobrze sobie radzi z cechami kategorialnymi oraz  z brakami danych, nie wykonam pierwszego kroku czyli EDA

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import os

data_path = "E:\page_ds\LightGBM_step_by_step"
pict_path = os.path.join(data_path, r"foto")

MainData = pd.read_csv(os.path.join(data_path, "train.csv"))

#podział na cechy oraz zmienną celu
y = MainData['target'].copy()
X = MainData.drop(columns=['id','target']).copy()

#lista cech kategorialnych
X_cat_col = list(X.loc[:,X.dtypes!=np.number].columns)
